In [2]:
# Mount google drive
from google.colab import drive
ROOT = "/content/drive"
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)

/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers
!pip install datasets
!pip install deepspeed
!pip install sentencepiece
!pip install sacrebleu

In [62]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer, DataCollatorForSeq2Seq
import sentencepiece
import torch
import numpy as np
import matplotlib.pyplot as plt
import re

In [4]:
%cd drive/MyDrive/Repos/llm-sparsification-cvf/

/content/drive/MyDrive/Repos/llm-sparsification-cvf


In [5]:
from src.exploration_utils import M2M100_layer_analysis, collect_params, plot_weight_distro, \
plot_weights_across_layers, weight_size_distro

In [6]:
%load_ext autoreload

In [7]:
%autoreload 2

In [8]:
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_1.2B")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_1.2B")

Downloading:   0%|          | 0.00/909 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/271 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

In [9]:
params = model.state_dict() 

In [10]:
len(params.keys())

1018

In [11]:
print(f"Total Parameters: {sum(p.numel() for p in model.parameters())}")

Total Parameters: 1239470080


In [12]:
# sample strings
hi_text = "जीवन एक चॉकलेट बॉक्स की तरह है।"
chinese_text = "生活就像一盒巧克力。"

In [13]:
# translate Hindi to French
tokenizer.src_lang = "hi"
encoded_hi = tokenizer(hi_text, return_tensors="pt")
generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.get_lang_id("fr"))
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


['La vie est comme une boîte de chocolat.']

In [14]:
# translate Chinese to English
tokenizer.src_lang = "zh"
encoded_zh = tokenizer(chinese_text, return_tensors="pt")
generated_tokens = model.generate(**encoded_zh, forced_bos_token_id=tokenizer.get_lang_id("en"))
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

['Life is like a box of chocolate.']

In [ ]:
# load dataset and metric
from datasets import load_dataset, load_metric
raw_datasets = load_dataset("wmt16", "de-en")
metric = load_metric("sacrebleu")

In [18]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 4548885
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2169
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2999
    })
})

In [84]:
from torch.nn.modules import padding
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "de"
tokenizer.tgt_lang = target_lang 
tokenizer.src_lang = source_lang
def preprocess_function(examples):
   inputs = [ex[source_lang] for ex in examples["translation"]]
   targets = [ex[target_lang] for ex in examples["translation"]]
   model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,
                            return_tensors="pt", padding='max_length')
   # Setup the tokenizer for targets
   #with tokenizer.as_target_tokenizer():
   labels = tokenizer(targets, max_length=max_target_length, truncation=True, padding='max_length',
                      return_tensors="pt")
   model_inputs["labels"] = labels["input_ids"]
   return model_inputs
tokenized_test_data = raw_datasets['test'].map(preprocess_function, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

In [120]:
small_eval_dataset = tokenized_test_data.shuffle(seed=42).select(range(1000))

In [121]:
small_eval_dataset

Dataset({
    features: ['translation', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1000
})

In [1]:
small_eval_dataset.features

NameError: ignored

In [ ]:
#small_eval_dataset.map(lambda example: print(example['input_ids']))

In [ ]:
pred_tokens = model.generate(torch.tensor(small_eval_dataset['input_ids']), forced_bos_token_id=tokenizer.get_lang_id("de"))

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [113]:
pred_tokens.shape

torch.Size([10, 43])

In [116]:
torch.tensor(small_eval_dataset['labels']).shape

torch.Size([10, 128])

In [117]:
metric.add_batch(predictions=pred_tokens, references=torch.tensor(small_eval_dataset['labels']))

In [118]:
final_score = metric.compute()

In [119]:
final_score

{'score': 0.300655860762251,
 'counts': [187, 0, 0, 0],
 'totals': [450, 440, 430, 420],
 'precisions': [41.55555555555556,
  0.11363636363636363,
  0.05813953488372093,
  0.02976190476190476],
 'bp': 1.0,
 'sys_len': 450,
 'ref_len': 10}

In [64]:
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

KeyError: ignored